# Figures for article on carbon monoxide ($CO$) emissions from plant leaves

In [ ]:
import pandas as pd
import numpy as np
import glob
from scipy import stats

from plotnine import *

In [ ]:
project_path = './'

input_fn = project_path + 'data_full.csv'

graphs_path  = project_path + 'graphs/'

# Colours
cbPalette = ['#000000', '#0072B2', '#E69F00', '#CC00CC', '#009E73', '#D55E00', '#CC79A7', '#FF3300', '#F0E442', '#56B4E9']
bwPalette = ['#ffffff', '#b0b0b0']

In [ ]:
def load_data(fn, silent=False):
    if (not silent): print('- All data')
    temp = pd.read_csv(fn, index_col=None)
    temp.rename({'date_mid_hour': 'timestamp'}, axis=1, inplace=True)
    temp['timestamp'] = pd.to_datetime(temp['timestamp'], format='%Y-%m-%d %H:%M:%S')
    if (not silent): print("    ", '100.0 %\t', fn.split('/')[-1])
    return(temp)

def bin_variable(data_column, steps):
    exponents = int(np.round(np.log10(steps)))
    # Determine minimum
    minimum = int(np.floor(min(data_column)))
    minimum = np.round(minimum, -exponents)-steps
    # Determine max and round
    maximum = int(np.ceil(max(data_column)) + 2*steps)
    maximum = np.round(maximum, -exponents)+steps
    #print(maximum, steps, np.round(maximum, -exponents)+steps)
    # Create list of cuts
    bins = list(np.arange(minimum, maximum, steps))
    # create labels
    if( (steps % 2) > 0):
        # If there are decimals, just convert to text without rounding
        bin_labels = [x+(steps/2) for x in bins[:-1]]
    else:
        # If there are NO decimals, round the values, then convert to text
        bin_labels = [str(int(x+(steps/2))) for x in bins[:-1]]
    if( ((steps % 2) > 0) and ((steps*100 % 2) == 0)):
        # If there are decimals, just convert to text without rounding
        bin_labels = [str(np.round(x+(steps/2),2)) for x in bins[:-1]]
    # Apply the bins
    bin_column = pd.cut(data_column, bins=bins, labels=bin_labels, include_lowest=True)
    return(bin_column)

# Order categorical columns
def order_cats(col, list_ordering):
    cat_dtype = pd.CategoricalDtype(categories=list_ordering, ordered=True)
    col = col.astype(cat_dtype)
    return(col)

def pvalue_text(p):
    if(p <= 0.001): p_text = '<.001'
    if(p > 0.001): p_text = '<.01'
    if(p > 0.01): p_text = '<.05'
    if(p > 0.05): p_text = p.round(2).astype(str)
    return(p_text)

### Data loading and figure creation

In [ ]:
print('Loading data...')

df = load_data(input_fn, silent=False)

print('Done...')

## Figure 2

In [ ]:
# Grid of midday CO & parameters thoughout the year (Figure 1)
#-------------------------------------------------------------

# Daily midday data
temp = df[['timestamp', 'treatment', 'season',
           'co.flux', 'co2.flux', 'Tr', 'TL',
           'PAR', 'VPD', 'co.ci', 'g_tCO']].copy()

# Remove some outliers
temp.loc[temp['co.flux'] < -0.8, 'co.flux'] = np.nan
temp.loc[temp['co.ci'] < 10, 'co.ci'] = np.nan
temp.loc[(temp['co.flux'] > 1) & (temp['timestamp'].dt.month < 4), 'co.flux'] = np.nan
temp.loc[(temp['co2.flux'] < -10) & (temp['timestamp'].dt.month >= 9), 'co2.flux'] = np.nan
temp.loc[temp['co2.flux'] > 2.5, 'co2.flux'] = np.nan

temp['doy'] = temp['timestamp'].dt.strftime('%j').astype(int)
midday = temp.loc[(temp['timestamp'].dt.hour >= 10) & (temp['timestamp'].dt.hour < 15)].copy()
midday.rename({'co.flux': 'flux_co'}, axis=1, inplace=True)
midday.rename({'co2.flux': 'flux_co2'}, axis=1, inplace=True)

# Show some stats
print("Stats")
print("-----")
print()
print('CO flux difference between droughted & irrigated:')
# t-tests Summer
a = midday.loc[(midday['season'] == 'Summer') & (midday['treatment'] == 'dro'),'flux_co']
b = midday.loc[(midday['season'] == 'Summer') & (midday['treatment'] == 'irr'),'flux_co']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

# t-tests Winter
a = midday.loc[(midday['season'] == 'Winter') & (midday['treatment'] == 'dro'),'flux_co']
b = midday.loc[(midday['season'] == 'Winter') & (midday['treatment'] == 'irr'),'flux_co']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

print('COi difference between winter & summer in droughted:')
a = midday.loc[(midday['season'] == 'Winter') & (midday['treatment'] == 'dro'),'co.ci']
b = midday.loc[(midday['season'] == 'Summer') & (midday['treatment'] == 'dro'),'co.ci']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

print('g_tCO difference between winter & summer in droughted:')
# t-tests Summer
a = midday.loc[(midday['season'] == 'Winter') & (midday['treatment'] == 'dro'),'g_tCO']
b = midday.loc[(midday['season'] == 'Summer') & (midday['treatment'] == 'dro'),'g_tCO']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

print('PAR difference between irr & dro in autumn:')
# t-tests Summer
a = midday.loc[(midday['season'] == 'Autumn') & (midday['treatment'] == 'irr'),'PAR']
b = midday.loc[(midday['season'] == 'Autumn') & (midday['treatment'] == 'dro'),'PAR']
t = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit') # Welch t-test for inequal variances
print('P =', pvalue_text(t[1]))

# Means
midday_means = midday.groupby(['season','treatment']).agg(['mean','median','std']).reset_index()
annual_means = midday.drop(columns='season').groupby(['treatment']).agg(['mean','median','std']).reset_index()

print('Midday means, by season')
display(midday_means)
print('Midday means, annual')
display(annual_means)

# Generate figure
#------------------
print('Generating figure...')
print('--------------------')

midday = midday.drop('season', axis=1)

midday2 = pd.melt(midday.loc[:, midday.columns != 'timestamp'], id_vars=['doy','treatment'])
grouped = midday2.groupby(['doy','treatment','variable']).agg(['mean','median','std']).reset_index()
grouped.columns = ['_'.join(col).strip('_') for col in grouped.columns.values]
grouped['timestamp'] = pd.to_datetime(grouped['doy'], format='%j')
grouped['treatment'] = grouped['treatment'].str.replace('dro', 'Droughted')
grouped['treatment'] = grouped['treatment'].str.replace('irr', 'Irrigated')


grouped.loc[grouped['variable'] == 'flux_co', 'variable'] = '$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]'
grouped.loc[grouped['variable'] == 'Tr', 'variable'] = '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]'
grouped.loc[grouped['variable'] == 'flux_co2', 'variable'] = '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]'
grouped.loc[grouped['variable'] == 'TL', 'variable'] = '$T_L$\n[$°C$]'
grouped.loc[grouped['variable'] == 'PAR', 'variable'] = '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]'
grouped.loc[grouped['variable'] == 'VPD', 'variable'] = '$VPD$\n[$Pa$]'
grouped.loc[grouped['variable'] == 'co.ci', 'variable'] = '$c_{CO,i}$\n[$nmol~mol^{-1}$]'
grouped.loc[grouped['variable'] == 'g_tCO', 'variable'] = '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'

grouped['variable'] = order_cats(grouped['variable'], list_ordering=['$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]',
                                                                     '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]',
                                                                     '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$T_L$\n[$°C$]',
                                                                     '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$VPD$\n[$Pa$]',
                                                                     '$c_{CO,i}$\n[$nmol~mol^{-1}$]',
                                                                     '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'])

point_df = grouped.copy()

#----------------
# Running average

# Try to do a running avg for Dan
grouped = midday.groupby(['doy','treatment']).agg(['mean']).reset_index()
grouped = grouped.loc[grouped['doy'] < 366].copy()
grouped.columns = ['_'.join(col).strip('_') for col in grouped.columns.values]
grouped.drop(['timestamp_mean'], axis=1, inplace=True)

# Make sure all doys are available
import itertools
doy_list = list(range(1, 366))
treatments = ['dro', 'irr']
combinations = list(itertools.product(doy_list, treatments))
time_df = pd.DataFrame(combinations, columns=['doy', 'treatment'])
grouped = time_df.merge(grouped, how='left', on=['doy','treatment'])
grouped['timestamp'] = pd.to_datetime(grouped['doy'], format='%j')

window_size = 14
cols =    ['flux_co_mean', 'flux_co2_mean', 'Tr_mean', 'TL_mean',
           'PAR_mean', 'VPD_mean', 'co.ci_mean', 'g_tCO_mean']
df_interpolated = grouped.groupby(['treatment'])[cols].transform(lambda s: s.rolling(window_size, min_periods=int(window_size/2)).mean())
df_temp = pd.concat([grouped[['doy', 'timestamp', 'treatment']], df_interpolated], axis=1)

graph_df = pd.melt(df_temp, id_vars=['timestamp','doy','treatment'])

graph_df['treatment'] = graph_df['treatment'].str.replace('dro', 'Droughted')
graph_df['treatment'] = graph_df['treatment'].str.replace('irr', 'Irrigated')

graph_df.loc[graph_df['variable'] == 'flux_co_mean', 'variable'] = '$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]'
graph_df.loc[graph_df['variable'] == 'Tr_mean', 'variable'] = '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]'
graph_df.loc[graph_df['variable'] == 'flux_co2_mean', 'variable'] = '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]'
graph_df.loc[graph_df['variable'] == 'TL_mean', 'variable'] = '$T_L$\n[$°C$]'
graph_df.loc[graph_df['variable'] == 'PAR_mean', 'variable'] = '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]'
graph_df.loc[graph_df['variable'] == 'VPD_mean', 'variable'] = '$VPD$\n[$Pa$]'
graph_df.loc[graph_df['variable'] == 'co.ci_mean', 'variable'] = '$c_{CO,i}$\n[$nmol~mol^{-1}$]'
graph_df.loc[graph_df['variable'] == 'g_tCO_mean', 'variable'] = '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'

graph_df['variable'] = order_cats(graph_df['variable'], list_ordering=['$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]',
                                                                     '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]',
                                                                     '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$T_L$\n[$°C$]',
                                                                     '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$VPD$\n[$Pa$]',
                                                                     '$c_{CO,i}$\n[$nmol~mol^{-1}$]',
                                                                     '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'
                                                                     ])

plt = ggplot(graph_df)
plt = plt + geom_point(aes(x='timestamp', y='value_mean', colour='treatment', linetype='treatment'), data=point_df, size=0.5)
plt = plt + geom_line(aes(x='timestamp', y='value', colour='treatment', linetype='treatment'), size=0.5)
#plt = plt + geom_ribbon(aes(x='timestamp', ymin='value_mean - value_std', ymax='value_mean + value_std', fill='treatment'), alpha=0.1)
plt = plt + scale_colour_manual(values=cbPalette) + scale_fill_manual(values=cbPalette)
plt = plt + theme_bw()
plt = plt + theme(axis_text_x=element_text(rotation = 90, hjust=0.5), axis_title_x = element_blank(), axis_title_y = element_blank(),
                  text=element_text(family="serif"),
                  legend_box_spacing=0.01, legend_title = element_blank(), legend_position='bottom')
plt = plt + theme(figure_size=(18/2.54, 27/2.54))
plt = plt + scale_x_datetime(date_breaks = '1 month', date_labels = '%b')
plt = plt + facet_grid('variable ~', scales='free_y')
print(plt)

plt.save(graphs_path + 'Fig2.png', width=19, height=27, units='cm', dpi=600)
plt.save(graphs_path + 'Fig2.pdf', width=19, height=27, units='cm', dpi=600)

## Figure 3: Boxplot

In [ ]:
temp = df.copy()
temp = temp.loc[(temp['timestamp'].dt.hour >= 10) & (temp['timestamp'].dt.hour < 15)].copy()
# Remove some bad values
temp.loc[temp['co.flux'] < -0.8, 'co.flux'] = np.nan
temp.loc[temp['co.ci'] < 10, 'co.ci'] = np.nan
temp.loc[(temp['co.flux'] > 1) & (temp['timestamp'].dt.month < 4), 'co.flux'] = np.nan
temp.loc[(temp['co2.flux'] < -10) & (temp['timestamp'].dt.month >= 9), 'co2.flux'] = np.nan
temp.loc[temp['co2.flux'] > 2.5, 'co2.flux'] = np.nan

temp['treatment'] = temp['treatment'].str.replace('dro', 'Droughted')
temp['treatment'] = temp['treatment'].str.replace('irr', 'Irrigated')

# PAR
temp2 = temp.copy()
temp2.dropna(subset=['co.flux','PAR'], inplace=True)
temp2['bins'] = bin_variable(temp2['PAR'], 500)
long_par = temp2[['treatment','season','co.flux','bins']].melt(id_vars=['treatment','season','bins'])
long_par['variable'] = '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]'

# T_leaf
temp2 = temp.copy()
temp2.dropna(subset=['co.flux','TL'], inplace=True)
temp2['bins'] = bin_variable(temp2['TL'], 10)
long_tl = temp2[['treatment','season','co.flux','bins']].melt(id_vars=['treatment','season','bins'])
long_tl['variable'] = '$T_L$\n[$°C$]'

# g_tCO total branch conductance
temp2 = temp.copy()
temp2['g_tCO'] = np.abs(temp2['g_tCO'])
temp2.dropna(subset=['co.flux','g_tCO'], inplace=True)
temp2['bins'] = bin_variable(temp2['g_tCO']*1000, 100)
long_gt = temp2[['treatment','season','co.flux','bins']].melt(id_vars=['treatment','season','bins'])
long_gt['variable'] = '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'

# Merge
long_merged = pd.concat([long_par, long_tl, long_gt])
long_merged['bins'] = long_merged['bins'].astype(float)
category_list = np.sort(long_merged['bins'].unique())
long_merged['bins'] = order_cats(long_merged['bins'], list_ordering=category_list) # Order bins

# Order seasons
long_merged['variable'] = order_cats(long_merged['variable'], list_ordering=['$PAR$\n[$\\mu mol~m^{-2}~s^{-1}$]', '$T_L$\n[$°C$]', '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'])

plt = ggplot(long_merged)
plt = plt + theme_bw()
plt = plt + geom_boxplot(aes(x='bins', y='value', fill='treatment'), outlier_size=0.5)
plt = plt + labs(x='', y='CO flux [$nmol~m^{-2}~s^{-1}$]', fill='')
plt = plt + facet_grid(' ~ variable', scales='free_x')
plt = plt + scale_colour_manual(values=bwPalette) + scale_fill_manual(values=bwPalette)
plt = plt + theme(axis_text_x=element_text(rotation = 90, hjust=0.5),
                  legend_title=element_blank(), #legend_box_spacing=10.4,
                  text=element_text(family="serif"))
plt = plt + theme(legend_position = 'bottom')
plt = plt + theme(figure_size=(18/2.54, 9/2.54))
plt = plt + coord_cartesian(ylim=[-0.5, 5])
print(plt)

plt.save(graphs_path + 'Fig3.png', width=18, height=9, units='cm', dpi=600)
plt.save(graphs_path + 'Fig3.pdf', width=18, height=9, units='cm', dpi=600)

## Figure 4: Diurnal figures in grids by season

In [ ]:
# Diurnals by season (Figure 4)
#------------------------------

temp = df[['timestamp', 'treatment','season',
           'co.flux', 'co2.flux', 'Tr', 'TL',
           'PAR', 'VPD', 'co.ci', 'g_tCO']].copy()

temp = temp.groupby(['treatment', 'season']).resample('1h', on='timestamp').mean()
temp = temp.reset_index()

# Convert wide to long
long = pd.melt(temp, id_vars=['timestamp','treatment','season'])
long['time']  = long['timestamp'].dt.strftime('%H:%M')
# Rename all columns
long.loc[long['variable'] == 'co.flux', 'variable'] = '$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]'
long.loc[long['variable'] == 'Tr', 'variable'] = '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]'
long.loc[long['variable'] == 'co2.flux', 'variable'] = '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]'
long.loc[long['variable'] == 'TL', 'variable'] = '$T_L$\n[$°C$]'
long.loc[long['variable'] == 'PAR', 'variable'] = '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]'
long.loc[long['variable'] == 'VPD', 'variable'] = '$VPD$\n[$Pa$]'
long.loc[long['variable'] == 'co.ci', 'variable'] = '$c_{CO,i}$\n[$nmol~mol^{-1}$]'
long.loc[long['variable'] == 'g_tCO', 'variable'] = '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'

long['treatment'] = long['treatment'].str.replace('dro', 'Droughted')
long['treatment'] = long['treatment'].str.replace('irr', 'Irrigated')

grouped = long.groupby(['time','season','treatment','variable']).agg(['mean','median','std'])
# Reset index and rename columns
grouped = grouped.reset_index()
grouped.columns = ['_'.join(col).strip('_') for col in grouped.columns.values]
grouped.columns = grouped.columns.str.replace('.', '_')

grouped['timestamp'] = pd.to_datetime(grouped['time'], format='%H:%M')

grouped['season'] = order_cats(grouped['season'], list_ordering=['Winter','Spring','Summer','Autumn'])
grouped['variable'] = order_cats(grouped['variable'], list_ordering=['$CO$ flux ($f_{CO}$)\n[$nmol~m^{-2}~s^{-1}$]',
                                                                     '$Tr$ flux\n[$mmol~m^{-2}~s^{-1}$]',
                                                                     '$CO_2$ flux\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$T_L$\n[$°C$]',
                                                                     '$PAR$\n[$\mu mol~m^{-2}~s^{-1}$]',
                                                                     '$VPD$\n[$Pa$]',
                                                                     '$c_{CO,i}$\n[$nmol~mol^{-1}$]',
                                                                     '$g_{t,CO}$\n[$mol~m^{-2}~s^{-1}$]'])

# Plot the diurnal
plt = ggplot(grouped)
plt = plt + geom_line(aes(x='timestamp', y='value_median', colour='treatment', linetype='treatment'), size=0.5)
plt = plt + geom_ribbon(aes(x='timestamp', ymin='value_median - value_std', ymax='value_median + value_std', fill='treatment'), alpha=0.1)
plt = plt + scale_colour_manual(values=cbPalette) + scale_fill_manual(values=cbPalette)
plt = plt + theme_bw()
plt = plt + labs(x='Time of day', y='Flux', colour='Treatment', fill='Treatment', linetype='Treatment')
plt = plt + theme(axis_text_x=element_text(rotation = 90, hjust=0.5), axis_title_x = element_blank(), axis_title_y = element_blank(),
                  text=element_text(family="serif"),
                  legend_box_spacing=0.01, legend_title = element_blank(), legend_position='bottom')
plt = plt + facet_grid('variable ~ season', scales='free_y')
plt = plt + theme(figure_size=(19/2.54, 27/2.54))
plt = plt + scale_x_datetime(date_breaks = '3 hours', date_labels = '%H:%M')
print(plt)

plt.save(graphs_path + 'Fig4.png', width=19, height=27, units='cm', dpi=600)
plt.save(graphs_path + 'Fig4.pdf', width=19, height=27, units='cm', dpi=600)